In [1]:
from settings import *

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
ppfeatures = [
    "vesselId", 
    "time",
    # 'time_diff',
    'cog',
    'sog',
    'rot',
    'heading',
    # 'navstat',
    # 'etaRaw',
    'latitude',
    'longitude',
]

In [4]:
input_features = ["vesselId", "time",
    'time_diff',
    'cog',
    'sog',
    'rot',
    'heading',
    # 'navstat',
    # 'etaRaw',
    'latitude',
    'longitude',
]

output_features = ["vesselId", "time",
    'cog',
    'sog',
    'rot',
    'heading',
    # 'navstat',
    # 'etaRaw',
    'latitude',
    'longitude',
]

In [5]:
ais_train = pd.read_csv(AIS_TRAIN, sep='|')
ais_train['time'] = pd.to_datetime(ais_train['time'])

In [6]:
# vessels = pd.read_csv(VESSELS, sep='|')
# vessels.head()

In [7]:
# pd.read_csv(SCHEDULES_TO_MAY_2024, sep="|")

In [8]:
ais_test = pd.read_csv(AIS_TEST, sep=",")
ais_test['time'] = pd.to_datetime(ais_test['time'])

In [9]:
# ais_test.head()

In [ ]:
len(ais_test["vesselId"].unique())

In [11]:
# pd.merge(ais_test, vessels, on="vesselId").dropna(subset="length")

In [12]:
# ais_test.sort_values(by=["vesselId", "time"]).iloc[:1]

In [13]:
# ais_test.value_counts(subset="vesselId")

In [14]:
# ais_test[ais_test["vesselId"]=="61e9f38eb937134a3c4bfd8d"]

In [15]:
# ais_test['time'] = pd.to_datetime(ais_test['time'])
# ais_test['time_diff'] = ais_test['time'].diff()

# mean_time_diff = ais_test['time_diff'].mean()
# std_diff = ais_test['time_diff'].std()
# print(mean_time_diff, std_diff)

In [16]:
# ais_test['time'] = pd.to_datetime(ais_test['time'])
# ais_test['time_diff'] = ais_test.sort_values(by=['vesselId', 'time']).groupby("vesselId")['time'].diff()

# mean_time_diff_per_id = ais_test.groupby('vesselId')['time_diff'].mean()

# print(mean_time_diff_per_id)

In [17]:
# mean_time_diff_per_id.describe()

In [18]:
# ais_test.sort_values(by=['vesselId', 'time'])

In [19]:

# def make_features(df: pd.DataFrame) -> pd.DataFrame:
#     raw = df.copy()

#     # time_diff: difference of time since last measure
#     raw['time'] = pd.to_datetime(raw['time'])
#     raw['time_diff'] = raw.sort_values(by=['vesselId', 'time']).groupby("vesselId")['time'].diff()
    
    
#     return raw

In [20]:
# ais_tr = make_features(ais_train)

In [21]:
# ais_tr.describe()

In [22]:
# te = ais_test.copy()

# # time_diff: difference of time since last measure
# te['time'] = pd.to_datetime(te['time'])
# te['time_diff'] = te.sort_values(by=['vesselId', 'time']).groupby("vesselId")['time'].diff()

In [23]:
# te.sort_values(by=["vesselId", "time"])

In [ ]:
ais_train["portId"].isna().sum()

In [25]:
# ais_train["etaRaw"].isna().sum()

In [26]:
# lbda = ais_train["etaRaw"].iloc[0]

In [27]:
# lbda.split()

In [28]:
# pd.to_datetime(ais_train["etaRaw"].apply(lambda x: str(x).split()))

In [ ]:
ais_train

In [30]:
# CREATE time_diff AND MAKE IT IN SECONDS

train_vessel_id_time = ais_train[ppfeatures].copy()
train_vessel_id_time["split"] = "train"
train_vessel_id_time["ID"] = train_vessel_id_time.index

test_vessel_id_time = ais_test[["ID", "vesselId", "time" ]].copy()
test_vessel_id_time["split"] = "test"
ais_data = pd.concat([train_vessel_id_time, test_vessel_id_time], ignore_index=True)

# arrival time diff (from etaRaw)
# 
ais_data['time_diff'] = (
    ais_data
    .sort_values(by=['time'])
    .groupby("vesselId")['time']
    .diff()
    .dropna()
    .dt.total_seconds()
    .astype(int)
)

def update_split_column(group: pd.Series) -> pd.Series:
    if group.name in ais_test["vesselId"].unique():
        group.iloc[-1] = "both"
    return group

df_temp = ais_data.copy()

ser_temp = pd.Series(
    df_temp[df_temp["split"]=="train"]
    .sort_values(by="time")
    .groupby("vesselId")["split"]
    .apply(update_split_column)
    .reset_index(drop=True)
)
ser_temp.index = df_temp[df_temp["split"]=="train"].sort_values(["vesselId", "time"]).index

ais_data["split"] = ser_temp

In [70]:
data_train = ais_data[(ais_data["split"]=="train")|(ais_data["split"]=="both")]
data_test = ais_data[(ais_data["split"]=="test")|(ais_data["split"]=="both")]

In [75]:
X = data_train.sort_values(["vesselId", "time"]).iloc[:-1]
y = data_train.sort_values(["vesselId", "time"]).iloc[1:]

In [40]:
scaler = StandardScaler()

In [41]:
# df_train = ais_train.drop(columns="portId")

In [ ]:
X

In [ ]:
y

In [43]:
X = X.drop(columns=["vesselId", "time"])
y = y.drop(columns=["vesselId", "time"])

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.1, shuffle=False)

In [45]:
import xgboost as xgb

In [46]:
params = {
    'n_estimators': 5000,
    'gamma': 0.5,
    'subsample': 0.6,
    'n_estimators': 5000,
    'min_child_weight':  15,
    'colsample_bytree': 0.8,
    'max_depth': 4,
    'eta':  0.005,
    'refresh_leaf': 1,
}

In [47]:
xgb_model = xgb.XGBRegressor(**params)

In [ ]:
xgb_model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    # early_stopping_rounds=50,
    verbose=False,
)

In [ ]:
results = xgb_model.evals_result()
train_errors = results['validation_0']['rmse']
test_errors = results['validation_1']['rmse']

plt.figure(figsize=(10, 6))
plt.plot(train_errors, label='Train')
plt.plot(test_errors, label='Test')
plt.xlabel('Boosting Rounds')
plt.ylabel("rmse")
plt.legend()
plt.title('Learning Curves')
plt.show()

In [ ]:
xgb_model.predict(X_test) 

In [ ]:
y_test

In [ ]:
ais_test.iloc[-100:]